In [1]:
import matplotlib
matplotlib.use("Agg")

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from cancernet.cancernet import CancerNet
from cancernet import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
NUM_EPOCHS=3; INIT_LR=1e-2; BS=32

trainPaths=list(paths.list_images(config.TRAIN_PATH))
lenTrain=len(trainPaths)
lenVal=len(list(paths.list_images(config.VAL_PATH)))
lenTest=len(list(paths.list_images(config.TEST_PATH)))

trainLabels=[int(p.split(os.path.sep)[-2]) for p in trainPaths]
trainLabels=to_categorical(trainLabels)
classTotals=trainLabels.sum(axis=0)
classWeight=classTotals.max()/classTotals

trainAug = ImageDataGenerator(
	rescale=1/255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.05,
	horizontal_flip=True,
	vertical_flip=True,
	fill_mode="nearest")

valAug=ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
	config.TRAIN_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)
valGen = valAug.flow_from_directory(
	config.VAL_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)
testGen = valAug.flow_from_directory(
	config.TEST_PATH,
	class_mode="categorical",
	target_size=(48,48),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 255815 images belonging to 2 classes.
Found 42660 images belonging to 2 classes.
Found 99906 images belonging to 2 classes.


In [3]:
model=CancerNet.build(width=48,height=48,depth=3,classes=2)
opt=Adagrad(lr=INIT_LR,decay=INIT_LR/NUM_EPOCHS)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])


M=model.fit_generator(
	trainGen,
	steps_per_epoch=lenTrain//BS,
	validation_data=valGen,
	validation_steps=lenVal//BS,
	epochs=NUM_EPOCHS)

C:\Users\arkap\anaconda3\lib\site-packages\keras\optimizer_v2\adagrad.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adagrad, self).__init__(name, **kwargs)
C:\Users\arkap\AppData\Local\Temp/ipykernel_15436/2089684941.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  M=model.fit_generator(


Epoch 1/3
7994/7994 [==============================] - 1831s 229ms/step - loss: 0.4352 - accuracy: 0.8253 - val_loss: 0.5104 - val_accuracy: 0.8012
Epoch 2/3
7994/7994 [==============================] - 1589s 199ms/step - loss: 0.4154 - accuracy: 0.8312 - val_loss: 0.5167 - val_accuracy: 0.7972
Epoch 3/3
7994/7994 [==============================] - 1434s 179ms/step - loss: 0.4111 - accuracy: 0.8328 - val_loss: 0.5120 - val_accuracy: 0.7990


In [4]:
print("Now evaluating the model")
testGen.reset()
pred_indices=model.predict_generator(testGen,steps=(lenTest//BS)+1)

pred_indices=np.argmax(pred_indices,axis=1)

print(classification_report(testGen.classes, pred_indices, target_names=testGen.class_indices.keys()))

cm=confusion_matrix(testGen.classes,pred_indices)
total=sum(sum(cm))
accuracy=(cm[0,0]+cm[1,1])/total
specificity=cm[1,1]/(cm[1,0]+cm[1,1])
sensitivity=cm[0,0]/(cm[0,0]+cm[0,1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')


Now evaluating the model


C:\Users\arkap\AppData\Local\Temp/ipykernel_15436/3850843823.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred_indices=model.predict_generator(testGen,steps=(lenTest//BS)+1)


              precision    recall  f1-score   support

           0       0.92      0.79      0.85     71451
           1       0.61      0.84      0.71     28455

    accuracy                           0.80     99906
   macro avg       0.77      0.81      0.78     99906
weighted avg       0.83      0.80      0.81     99906

[[56104 15347]
 [ 4564 23891]]
Accuracy: 0.8007026605008708
Specificity: 0.8396063960639606
Sensitivity: 0.7852094442345103


In [5]:
from tensorflow.keras.models import load_model, model_from_json


fmodel_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(fmodel_json)
print('Model Saved')
model.save_weights('model1.h5')
print('Weights saved')

Model Saved
Weights saved
